In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import math

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.decomposition import RandomizedPCA
from sklearn.metrics import classification_report
from sklearn import preprocessing
import matplotlib.pyplot as plt
import scipy.optimize as opt
import glob
import os
import sys
import pickle
import time
import timeit

from sklearn.decomposition import RandomizedPCA

import theano
import theano.tensor as T
import lasagne

#np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Layer, Reshape, AutoEncoder, Merge
from keras.layers.convolutional import Convolution2D,MaxPooling2D,Convolution1D,MaxPooling1D,UpSampling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import keras.utils.visualize_util as vutil
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l1, l2, l1l2, activity_l2, activity_l1
import keras.layers.containers as containers
from keras.layers.noise import GaussianNoise
from keras.regularizers import ActivityRegularizer
from keras import backend as K
import keras.utils.visualize_util as vutil
from keras.models import model_from_json

import seaborn as sns

%matplotlib inline

from IPython.display import SVG

import h5py

np.set_printoptions(suppress=True)  #Supress exponent of the number 2e-2 == 0.02

%matplotlib inline

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: no CUDA-capable device is detected)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
Using Theano backend.


In [2]:
#CPCG0235

In [3]:
#Read the data

GenomicFeatures_CPCG0235 = pd.read_csv(open('data/CPCG0235/GenomicFeatures_CPCG0235.txt','r'),sep='\t')
SNVCalls_CPCG0235    = pd.read_csv(open('data/CPCG0235/SNVCalls_CPCG0235.txt','r'),sep='\t') 

In [4]:
print GenomicFeatures_CPCG0235.shape
print SNVCalls_CPCG0235.shape

(130910, 15)
(130910, 146)


In [5]:
GenomicFeatures_CPCG0235.head()

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,10051,740,23,20653,959,1024,15,39.0,GTT,37.880000,0.378109,NaN,41428,0.775808
1,chr1,16288,40,12,1563,56,57,23,64.5,ACA,2.468966,0.492537,ncRNA_intronic,35191,0.589286
2,chr1,16497,116,55,4939,179,113,23,55.0,TTC,2.797101,0.422886,ncRNA_intronic,34982,0.547486
3,chr1,19925,141,16,4669,171,108,0,56.0,GCA,2.376712,0.572139,ncRNA_intronic,31554,0.643275
4,chr1,48518,116,17,4067,149,129,0,41.0,TTA,2.810219,0.383085,intergenic,2961,0.496644


In [6]:
SNVCalls_CPCG0235.head()

,CHROM,POS,END,X2399006,X2673217,X2673229,X2679064,X2679067,X2706568,X2762069,X2762071,X2788863,X2788866,X2788869,X2788872,X2788945,X2788949,X2788953,X2788956,X2791419,
0,chr1,10050,10051,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,chr1,16287,16288,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
2,chr1,16496,16497,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
3,chr1,19924,19925,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,...
4,chr1,48517,48518,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...


In [7]:
np.unique(GenomicFeatures_CPCG0235['Trinucleotide'])

array(['ACA', 'ACC', 'ACG', 'ACT', 'ATA', 'ATC', 'ATG', 'ATT', 'CCA',
       'CCC', 'CCG', 'CCT', 'CTA', 'CTC', 'CTG', 'CTT', 'GCA', 'GCC',
       'GCG', 'GCT', 'GTA', 'GTC', 'GTG', 'GTT', 'TCA', 'TCC', 'TCG',
       'TCT', 'TTA', 'TTC', 'TTG', 'TTT'], 
      dtype='|S3')

In [8]:
CPCG0235 = pd.merge(GenomicFeatures_CPCG0235, SNVCalls_CPCG0235,left_on=['CHROM', 'POS'], right_on=['CHROM','END'])


In [9]:
CPCG0235.head() #182+15

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,END,X2399006,X2673217,X2673229,
0,chr1,10051,740,23,20653,959,1024,15,39.0,GTT,37.880000,0.378109,NaN,41428,0.775808,10050,10051,0,0,0,...
1,chr1,16288,40,12,1563,56,57,23,64.5,ACA,2.468966,0.492537,ncRNA_intronic,35191,0.589286,16287,16288,0,0,0,...
2,chr1,16497,116,55,4939,179,113,23,55.0,TTC,2.797101,0.422886,ncRNA_intronic,34982,0.547486,16496,16497,0,0,0,...
3,chr1,19925,141,16,4669,171,108,0,56.0,GCA,2.376712,0.572139,ncRNA_intronic,31554,0.643275,19924,19925,1,0,0,...
4,chr1,48518,116,17,4067,149,129,0,41.0,TTA,2.810219,0.383085,intergenic,2961,0.496644,48517,48518,0,0,0,...


In [10]:
CPCG0235_X = CPCG0235.iloc[:,2:15] #Exclude CHROM, POS
CPCG0235_Y = CPCG0235.iloc[:,17:]  #Exclude CHROM_y, POS_y, END

In [11]:
print CPCG0235_X.shape
print CPCG0235_Y.shape

(130910, 13)
(130910, 143)


In [12]:
CPCG0235_X[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,740,23,20653,959,1024,15,39.0,GTT,37.880000,0.378109,NaN,41428,0.775808
1,40,12,1563,56,57,23,64.5,ACA,2.468966,0.492537,ncRNA_intronic,35191,0.589286


In [13]:
CPCG0235_Y[:2]

,X2399006,X2673217,X2673229,X2679064,X2679067,X2706568,X2762069,X2762071,X2788863,X2788866,X2788869,X2788872,X2788945,X2788949,X2788953,X2788956,X2791419,X2791423,X2791429,X2791432,
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...


In [14]:
totalSample = CPCG0235_Y.shape[0]
CPCG0235_Y_Val = np.zeros((totalSample,1)) #Will be our label now

print totalSample
print CPCG0235_Y_Val.shape

130910
(130910, 1)


In [15]:
CPCG0235_Y.shape

(130910, 143)

In [16]:
CPCG0235_Y.values[0].shape

(143,)

In [17]:
for i in range(totalSample):
    CPCG0235_Y_Val[i] = np.argmax(np.bincount(CPCG0235_Y.values[i]))

In [18]:
CPCG0235_Y_Val_1 = pd.DataFrame(CPCG0235_Y_Val)
CPCG0235_Y_Val_1.columns = ['Label']

In [19]:
CPCG0235_Y_Val_1[:3]

,Label
0,0
1,0
2,0


In [20]:
CPCG0235_dataframe = pd.concat([CPCG0235_X, CPCG0235_Y_Val_1],axis=1)

In [21]:
CPCG0235_dataframe.shape

(130910, 14)

In [22]:
CPCG0235_dataframe.to_csv('CPCG0235.csv')

In [23]:
#Now we have variable importance ...lets do analysis

In [24]:
%load_ext rpy2.ipython

In [25]:
%%R
# Read in the data
CPCG0235Frame = read.csv("CPCG0235.csv")
str(CPCG0235Frame)

'data.frame':	130910 obs. of  15 variables:
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  740 40 116 141 116 89 52 67 43 90 ...
 $ NonRefAllele   : int  23 12 55 16 17 25 23 11 23 20 ...
 $ BaseQual       : int  20653 1563 4939 4669 4067 3435 1969 2382 2017 3249 ...
 $ TumourCoverage : int  959 56 179 171 149 130 87 83 72 120 ...
 $ NormalCoverage : int  1024 57 113 108 129 87 31 56 84 99 ...
 $ MapQual        : num  15 23 23 0 0 0 15 0 23 46 ...
 $ ReadPosition   : num  39 64.5 55 56 41 49 43 45 49.5 59.5 ...
 $ Trinucleotide  : Factor w/ 32 levels "ACA","ACC","ACG",..: 24 1 30 17 29 7 15 8 2 32 ...
 $ HomopolymerRate: num  37.88 2.47 2.8 2.38 2.81 ...
 $ GCcontent      : num  0.378 0.493 0.423 0.572 0.383 ...
 $ GenomicLocation: Factor w/ 16 levels "","downstream",..: 1 8 8 8 5 5 5 5 5 5 ...
 $ DistanceSNP    : int  41428 35191 34982 31554 2961 2500 324 2019 8007 10792 ...
 $ StrandBias     : num  0.776 0.589 0.547 0.643 0.497 ...
 $ Label          : num  

In [26]:
%%R
#sum(is.na(CPCG0183Frame$StrandBias))
sum(is.na(CPCG0235Frame$ReadPosition)) 

[1] 24


In [27]:
%%R
CPCG0235Frame$ReadPosition[is.na(CPCG0235Frame$ReadPosition)] = mean(CPCG0235Frame$ReadPosition, na.rm=TRUE)
CPCG0235Frame$StrandBias[is.na(CPCG0235Frame$StrandBias)] = mean(CPCG0235Frame$StrandBias, na.rm=TRUE)

print (sum(is.na(CPCG0235Frame$ReadPosition)) )
print (sum(is.na(CPCG0235Frame$StrandBias)) )

[1] 0
[1] 0


In [28]:
%%R
sum(is.na(CPCG0235Frame))  #No NA value now

[1] 0


In [29]:
%%R
write.csv(CPCG0235Frame, file = "CPCG0235_imputed.csv")  #Cleaned CSV without NA

In [30]:
%%R
library(h2o)
localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE,min_mem_size = "3g")

/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: statmod

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘h2o’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         1 hours 3 minutes 
    H2O cluster version:        3.9.1.3435 
    H2O cluster name:           H2O_started_from_R_rbhat_szm488 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   26.56 GB 
    H2O cluster total cores:    56 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.0 beta (2016-03-30 r70404) 



In [31]:
%%R
## Import CSV as H2O:
csvPath = "/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/CPCG0235_imputed.csv"
CPCG0235.hex          = h2o.importFile(path = csvPath, destination_frame = "CPCG0235.hex") 
train                 = as.data.frame(CPCG0235.hex)

#train  = subset(train,C1<20000)

train$Trinucleotide   = as.factor(train$Trinucleotide)
train$GenomicLocation = as.factor(train$GenomicLocation)
train$Label           = as.factor(train$Label)

train_h2o             = as.h2o(train)

  |======================================================================| 100%
  |======================================================================| 100%


In [32]:
%%R
#Training a deep learning model:---------------------------------------------------------#
model <- h2o.deeplearning( x = 3:15, # column numbers for predictors
                           y = 16,            # column number for label
                           training_frame = train_h2o,
                           activation = "RectifierWithDropout", 
                           input_dropout_ratio = 0.2,
                           hidden_dropout_ratios = c(0.5,0.5),
                           balance_classes = TRUE, 
                           hidden = c(10,8),  ## two hidden layers
                           epochs = 50,
                           variable_importances = T)

  |======================================================================| 100%


In [33]:
%%R
var_importance = h2o.varimp(model)
#print(var_importance)
h2o.varimp(model)

Variable Importances: 
        variable relative_importance scaled_importance percentage
1   NonRefAllele            1.000000          1.000000   0.274644
2 TumourCoverage            0.367006          0.367006   0.100796
3      RefAllele            0.283771          0.283771   0.077936
4 NormalCoverage            0.263137          0.263137   0.072269
5       BaseQual            0.225915          0.225915   0.062046

---
                     variable relative_importance scaled_importance percentage
55          Trinucleotide.CCG            0.008136          0.008136   0.002234
56   GenomicLocation.intronic            0.007924          0.007924   0.002176
57          Trinucleotide.TTG            0.007101          0.007101   0.001950
58                 StrandBias            0.005573          0.005573   0.001531
59 GenomicLocation.intergenic            0.003395          0.003395   0.000932
60  Trinucleotide.missing(NA)            0.000000          0.000000   0.000000


In [34]:
%%R
write.table(var_importance, file="CPCG0235_variable_importance.csv")

In [35]:
var_importance = pd.read_csv(open('CPCG0235_variable_importance.csv','r'),sep=' ')
#pd.read_csv('CPCG0100_variable_importance.csv')

In [36]:
var_importance[:5]

,variable,relative_importance,scaled_importance,percentage
1,NonRefAllele,1.000000,1.000000,0.274644
2,TumourCoverage,0.367006,0.367006,0.100796
3,RefAllele,0.283771,0.283771,0.077936
4,NormalCoverage,0.263137,0.263137,0.072269
5,BaseQual,0.225915,0.225915,0.062046


In [37]:
GenomicFeatures_CPCG0235[:5]

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,10051,740,23,20653,959,1024,15,39.0,GTT,37.880000,0.378109,NaN,41428,0.775808
1,chr1,16288,40,12,1563,56,57,23,64.5,ACA,2.468966,0.492537,ncRNA_intronic,35191,0.589286
2,chr1,16497,116,55,4939,179,113,23,55.0,TTC,2.797101,0.422886,ncRNA_intronic,34982,0.547486
3,chr1,19925,141,16,4669,171,108,0,56.0,GCA,2.376712,0.572139,ncRNA_intronic,31554,0.643275
4,chr1,48518,116,17,4067,149,129,0,41.0,TTA,2.810219,0.383085,intergenic,2961,0.496644


In [38]:
var_imp_dict = {}

for i in range(var_importance.shape[0]):
    var_imp_dict[var_importance.iloc[i,0]] = var_importance.iloc[i,1]

var_imp_dict

{'BaseQual': 0.22591510415077201,
 'DistanceSNP': 0.081409119069576305,
 'GCcontent': 0.013976084068417501,
 'GenomicLocation.UTR3': 0.0137208290398121,
 'GenomicLocation.UTR5': 0.032354906201362603,
 'GenomicLocation.UTR5;UTR3': 0.046582501381635701,
 'GenomicLocation.downstream': 0.0225263833999634,
 'GenomicLocation.exonic': 0.020142843946814499,
 'GenomicLocation.exonic;splicing': 0.058884296566247898,
 'GenomicLocation.intergenic': 0.0033947108313441303,
 'GenomicLocation.intronic': 0.0079244719818234409,
 'GenomicLocation.missing(NA)': 0.0183493364602327,
 'GenomicLocation.ncRNA_UTR3': 0.0283181127160788,
 'GenomicLocation.ncRNA_UTR5': 0.0436825230717659,
 'GenomicLocation.ncRNA_exonic': 0.029515836387872699,
 'GenomicLocation.ncRNA_intronic': 0.028384642675519,
 'GenomicLocation.splicing': 0.043282497674226802,
 'GenomicLocation.upstream': 0.0208651199936867,
 'GenomicLocation.upstream;downstream': 0.032065566629171399,
 'HomopolymerRate': 0.049229618161916698,
 'MapQual': 0.224

In [39]:
noOfSample = GenomicFeatures_CPCG0235.shape[0]

CPCG0235_Dict = {}

for i in range(noOfSample):
    
    pos = GenomicFeatures_CPCG0235.iloc[i,1] #gets the pos value
    
    RefAllele      = GenomicFeatures_CPCG0235.iloc[i,2]*var_imp_dict['RefAllele']
    NonRefAllele   = GenomicFeatures_CPCG0235.iloc[i,3]*var_imp_dict['NonRefAllele']
    BaseQual       = GenomicFeatures_CPCG0235.iloc[i,4]*var_imp_dict['BaseQual']
    TumourCoverage = GenomicFeatures_CPCG0235.iloc[i,5]*var_imp_dict['TumourCoverage']
    NormalCoverage = GenomicFeatures_CPCG0235.iloc[i,6]*var_imp_dict['NormalCoverage']
    MapQual        = GenomicFeatures_CPCG0235.iloc[i,7]*var_imp_dict['MapQual']
    ReadPosition   = GenomicFeatures_CPCG0235.iloc[i,8]*var_imp_dict['ReadPosition']
    
    Trinucleotide  = 0
    if isinstance(GenomicFeatures_CPCG0235.iloc[i,9], str):
        Trinucleotide  = var_imp_dict['Trinucleotide.'+str(GenomicFeatures_CPCG0235.iloc[i,9])]
    HomopolymerRate= GenomicFeatures_CPCG0235.iloc[i,10]*var_imp_dict['HomopolymerRate']
    GCcontent      = GenomicFeatures_CPCG0235.iloc[i,11]*var_imp_dict['GCcontent']
    
    GenomicLocation = 0
    if isinstance(GenomicFeatures_CPCG0235.iloc[i,12], str):
        GenomicLocation= var_imp_dict['GenomicLocation.'+str(GenomicFeatures_CPCG0235.iloc[i,12])]
    DistanceSNP    = GenomicFeatures_CPCG0235.iloc[i,13]*var_imp_dict['DistanceSNP']
    StrandBias     = GenomicFeatures_CPCG0235.iloc[i,14]*var_imp_dict['StrandBias']
    
    #print 'RefAllele=',RefAllele,' NonRefAllele=',NonRefAllele,' BaseQual=',BaseQual,' TumourCoverage=',TumourCoverage
    #print 'NormalCoverage=',NormalCoverage,' MapQual=',MapQual,' ReadPosition=',ReadPosition,' Trinucleotide=',Trinucleotide
    #print 'HomopolymerRate=',HomopolymerRate,' GCcontent=',GCcontent,' GenomicLocation=',GenomicLocation,' DistanceSNP=',DistanceSNP
    #print 'StrandBias=',StrandBias
    CPCG0235_Dict[pos] = RefAllele+NonRefAllele+BaseQual+TumourCoverage+NormalCoverage+MapQual+ReadPosition+Trinucleotide+ \
                         HomopolymerRate+GCcontent+GenomicLocation+DistanceSNP+StrandBias

In [40]:
#Sorting dictionary
CPCG0235_Dict1 = [(k,v) for v,k in sorted([(v,k) for k,v in CPCG0235_Dict.items()],reverse=True)]

In [41]:
CPCG0235_Dict_Frame = pd.DataFrame(CPCG0235_Dict1, columns=['POS', 'Value'])

In [42]:
CPCG0235_Dict_Frame.to_csv("CPCG0235_Dict_Frame.csv")

In [22]:
CPCG0235_Dict_Frame = pd.read_csv("CPCG0235_Dict_Frame.csv")

In [43]:
CPCG0235_Dict_Frame[:10]

,POS,Value
0,58884756,2518958.279306
1,58854511,2518924.016174
2,58866141,2508525.411544
3,58829547,2504675.648524
4,58834058,2502802.586659
5,58855088,2496613.505358
6,58912787,2484375.450973
7,58876064,2483735.868718
8,58869542,2482079.753849
9,58876033,2481630.571069


In [23]:
GenomicFeatures_CPCG0235_1 = pd.concat([CPCG0235.iloc[:,0:16], CPCG0235_Y_Val_1],axis=1)

In [25]:
rows = CPCG0235_Dict_Frame['POS'][:10].values
print rows
GenomicFeatures_CPCG0235_Top10 = GenomicFeatures_CPCG0235_1.loc[GenomicFeatures_CPCG0235_1['POS_x'].isin(rows)]

[58884756 58854511 58866141 58829547 58834058 58855088 58912787 58876064
 58869542 58876033]


In [27]:
GenomicFeatures_CPCG0235_Top10[:10]

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,Label
130621,chrY,58829547,6227,952,220917,9939,10626,0,59,GCA,2.921260,0.373134,NaN,30040904,0.558306,58829546,0
130623,chrY,58834058,6908,194,214607,8649,11476,0,40,CTA,2.227586,0.368159,NaN,30045415,0.201064,58834057,0
130637,chrY,58854511,7369,675,238656,20338,27139,0,45,GTA,2.555556,0.353234,NaN,30065868,0.855984,58854510,0
130638,chrY,58855088,6042,76,184587,8328,9052,0,60,ACA,2.518248,0.378109,NaN,30066445,0.180956,58855087,0
130651,chrY,58866141,6020,1098,221715,9960,12889,0,57,TCT,2.222222,0.368159,NaN,30077498,0.351205,58866140,0
130657,chrY,58869542,3980,80,124746,5762,6512,0,42,CTT,2.639098,0.407960,NaN,30080899,0.463901,58869541,0
130661,chrY,58876033,4018,69,120905,5653,6247,0,49,GTT,2.456522,0.388060,NaN,30087390,0.399788,58876032,0
130662,chrY,58876064,4105,114,129026,5879,6686,0,47,GCA,2.427536,0.388060,NaN,30087421,0.430856,58876063,0
130674,chrY,58884756,8007,54,244875,16080,20182,0,80,CTT,2.559701,0.393035,NaN,30096113,0.528483,58884755,0
130691,chrY,58912787,3428,379,118887,5711,6417,0,54,GCT,2.352518,0.398010,NaN,30124144,0.147260,58912786,0
